In [0]:
-- create new managed volume LANDING under dev > bronze

CREATE VOLUME IF NOT EXISTS dev.bronze.landing
COMMENT 'This is Landing Managed Volume'
;


In [0]:
%python
# create new folder called input under LANDING volume

dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/input")


In [0]:
%python
# copy retail invoice data from databricks-datasets
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/input/")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/input/")

In [0]:
-- create placeholder table dev.bronze.invoice_cp
CREATE TABLE IF NOT EXISTS dev.bronze.invoice_cp;

In [0]:
--use COPY INTO to load data into placeholder table
COPY INTO dev.bronze.invoice_cp
FROM "/Volumes/dev/bronze/landing/input"
FILEFORMAT = CSV
PATTERN = '*.csv'
FORMAT_OPTIONS (
  'mergeSchema' = 'true',
  'header' = 'true'
)
COPY_OPTIONS (
  'mergeSchema' = 'true'
)
;

In [0]:
--view data from table
select * from dev.bronze.invoice_cp

In [0]:
describe table extended dev.bronze.invoice_cp

In [0]:
-- create new table with only 3 columns invoice_cp_alt and custom columns insert_date
CREATE TABLE dev.bronze.invoice_cp_alt(
  InvoiceNo STRING,
  StockCode STRING,
  Quantity DOUBLE,
  _insert_date TIMESTAMP
)
;

In [0]:
-- load data using COPY INTO in the new table invoice_cp_alt
COPY INTO dev.bronze.invoice_cp_alt
FROM (
  SELECT InvoiceNo, StockCode, cast(Quantity as double) Quantity, current_timestamp() as _insert_date
  FROM 
    "/Volumes/dev/bronze/landing/input"
)
FILEFORMAT = CSV
PATTERN = '*.csv'
FORMAT_OPTIONS (
  'mergeSchema' = 'true',
  'header' = 'true'
)
;


In [0]:
select * from dev.bronze.invoice_cp_alt

In [0]:
%python
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv", "/Volumes/dev/bronze/landing/input/")